# 목표
1. 장르가 최대한 뒤섞인 url 1000개를 크롤링 후 NLP 알고리즘을 이용해 장르 맞추기
2. 1번 완료 후 각 게임별 설명도 추가해 정확도 높이기
3. 1,2과정을 labeling을 이용해 해결하면 이후 unsupervised-learning을 이용해 나눠보고 특징 찾기

training_set은 뒤섞인 1000개를 쓰고, test_set은 사전 예약 게임(아직 출시x)로 쓰기

In [1]:
import urllib.request
import time
import numpy as np
import csv
import pandas as pd

from urllib.request import urlopen
from urllib.parse import quote_plus
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
soup = []
url = []
title = []
url_href = []
title_genre = []
url_genre = []
number = 8

1. url : url을 크롤링 한 것
2. title : url 중 class가 같은 것 수집
3. url_href : class가 같아 수집한 내용들의 href를 수집
4. number : url 개수

In [3]:

# 인기 앱/게임
url_0 = 'https://play.google.com/store/apps/collection/cluster?clp=0g4cChoKFHRvcHNlbGxpbmdfZnJlZV9HQU1FEAcYAw%3D%3D:S:ANO1ljJ_Y5U&gsr=Ch_SDhwKGgoUdG9wc2VsbGluZ19mcmVlX0dBTUUQBxgD:S:ANO1ljL4b8c'
url.append(url_0)

# 인기 유로 게임
url_1 = 'https://play.google.com/store/apps/collection/cluster?clp=0g4cChoKFHRvcHNlbGxpbmdfcGFpZF9HQU1FEAcYAw%3D%3D:S:ANO1ljLtt38&gsr=Ch_SDhwKGgoUdG9wc2VsbGluZ19wYWlkX0dBTUUQBxgD:S:ANO1ljJCqyI'
url.append(url_1)

# 최고 매출 게임
url_2 = 'https://play.google.com/store/apps/collection/cluster?clp=0g4YChYKEHRvcGdyb3NzaW5nX0dBTUUQBxgD:S:ANO1ljLhYwQ&gsr=ChvSDhgKFgoQdG9wZ3Jvc3NpbmdfR0FNRRAHGAM%3D:S:ANO1ljIKta8&hl=ko'
url.append(url_2)

# 금주의 신규 추천 게임
url_3 = 'https://play.google.com/store/apps/collection/cluster?clp=SjYKKgokcHJvbW90aW9uXzMwMDA3OTFfbmV3X3JlbGVhc2VzX2dhbWVzEEoYAxIER0FNRToCCAI%3D:S:ANO1ljJYK2k&gsr=CjhKNgoqCiRwcm9tb3Rpb25fMzAwMDc5MV9uZXdfcmVsZWFzZXNfZ2FtZXMQShgDEgRHQU1FOgIIAg%3D%3D:S:ANO1ljICr10'
url.append(url_3)

# 맞춤 추천 (이건 내 아이디에 맞춰져 추천되니 장르가 많지 않을 수 있음)
url_4 = 'https://play.google.com/store/apps/collection/cluster?clp=ogoQCAESBEdBTUUqAggCUgIIAQ%3D%3D:S:ANO1ljJlEdM&gsr=ChOiChAIARIER0FNRSoCCAJSAggB:S:ANO1ljJdubc'
url.append(url_4)

# 최근 신규 추천 게임
url_5 = 'https://play.google.com/store/apps/collection/cluster?clp=Sj8KMwotcHJvbW90aW9uXzMwMDFhMDNfbmV3X3JlbGVhc2VzX2dhbWVzX2NhdGVnb3J5EEoYAxIER0FNRToCCAI%3D:S:ANO1ljL0zR8&gsr=CkFKPwozCi1wcm9tb3Rpb25fMzAwMWEwM19uZXdfcmVsZWFzZXNfZ2FtZXNfY2F0ZWdvcnkQShgDEgRHQU1FOgIIAg%3D%3D:S:ANO1ljKuCNU&hl=ko'
url.append(url_5)

# 인스턴트 게임
url_6 = 'https://play.google.com/store/apps/collection/cluster?clp=SjAKJAoecHJvbW90aW9uXzMwMDMwOGZfaW5zdGFudGdhbWVzEEoYAxIER0FNRToCCAI%3D:S:ANO1ljLWnrU&gsr=CjJKMAokCh5wcm9tb3Rpb25fMzAwMzA4Zl9pbnN0YW50Z2FtZXMQShgDEgRHQU1FOgIIAg%3D%3D:S:ANO1ljK-g4E&hl=ko'
url.append(url_6)

# 에디터 추천 게임
url_7 = 'https://play.google.com/store/apps/collection/cluster?clp=ogouCAESBEdBTUUaHAoWcmVjc190b3BpY19vOVU4RlFsZzgtNBA7GAMqAggCUgIIAg%3D%3D:S:ANO1ljJ_Ozc&gsr=CjGiCi4IARIER0FNRRocChZyZWNzX3RvcGljX285VThGUWxnOC00EDsYAyoCCAJSAggC:S:ANO1ljIOr3E&hl=ko'
url.append(url_7)


1. url_0, url_1, url_2는 각각 인기 앱, 인기 유료, 최고 매출 게임이고 장르가 섞여 있습니다.각 개수는 200개 씩이며, 중복되는게 있습니다.
(총 600개 -> 중복 제거 : 533개)
2. 약 500개를 더 보강해야 하지만, 장르가 섞여 있으면 좋겠다 (url_7까지 하니 833개)

In [4]:
for i in range(number):
    driver = webdriver.Chrome()
    driver.get(url[i])
    driver.implicitly_wait(3) # 3초간 기다림

    cnt = 0
    while(cnt<10): # 스크롤을 10번 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        cnt +=1
        time.sleep(0.5)
    html = driver.page_source
    driver.quit()
    soup.append(BeautifulSoup(html, 'html.parser'))

스크롤을 내리면 새로운 내용이 추가되기 때문에 while문을 이용해 스크롤 내리는 행동을 해야함 

In [5]:
for i in range(number):
    title.append(soup[i].find_all(class_ = 'poRVub' ))
    # url_href : 우리가 필요한 href
    for i in title[i]:
        url_href.append(i.attrs['href'])

In [6]:
len(url_href)

1045

In [7]:
url_href

['/store/apps/details?id=com.sundaytoz.kakao.anipang4',
 '/store/apps/details?id=com.netmarble.ma9bkr',
 '/store/apps/details?id=com.gravity.ragnarokorigin.aos',
 '/store/apps/details?id=com.jhulse.lineup',
 '/store/apps/details?id=com.redforcegames.stack.colors',
 '/store/apps/details?id=com.nexon.kart',
 '/store/apps/details?id=com.xtt.iasurv2',
 '/store/apps/details?id=ru.galya.drawjoust',
 '/store/apps/details?id=com.dangernoodle.killerbee',
 '/store/apps/details?id=com.monawa.samblood',
 '/store/apps/details?id=com.sangoku2kr.android',
 '/store/apps/details?id=com.idleif.abyssrium',
 '/store/apps/details?id=com.hellotoon.shinvatar',
 '/store/apps/details?id=com.klab.bleach',
 '/store/apps/details?id=com.outfit7.mytalkingtomfriends',
 '/store/apps/details?id=com.ncsoft.lineagem19',
 '/store/apps/details?id=com.cg.moneybuster',
 '/store/apps/details?id=com.yoozoo.kr.snsgz2',
 '/store/apps/details?id=droidhang.twgame.restaurant',
 '/store/apps/details?id=com.neonnewcrystal.blast',
 '

여기 까지 600개 게임 정보를 모았음, 중복 제거 필요

# 게임 제목 가져오기

In [8]:
game_title = []
imsi_title = []
for i in range(number):
    imsi_title.append(soup[i].find_all(class_ = 'WsMG1c nnK0zc' ))
    # url_href : 우리가 필요한 href
    for i in imsi_title[i]:
        game_title.append(i.text)

In [9]:
len(game_title)

1045

In [10]:
game_title

['애니팡4',
 '마구마구 2020',
 '라그나로크 오리진',
 '라인업: 몽타주 그리기',
 'Stack Colors!',
 '카트라이더 러쉬플러스',
 'Idle Arks: Build at Sea',
 'Draw Joust!',
 'Murder Hornet',
 '삼국지혈전',
 '진삼국대전2',
 '어비스리움 - 나만의 힐링 수족관 (방치형 시뮬레이션, VR 지원)',
 '신바타 : 신비아파트 스타일 캐릭터 만들기',
 'BLEACH Brave Souls - 3D 액션',
 '마이토킹톰 프렌즈',
 '리니지M',
 'Money Buster',
 '그랑삼국',
 '고양이 식탁',
 '크리스탈 블래스트',
 '클래시 붐(Clash Boom)',
 '배틀그라운드',
 '궁3D',
 '걸스 X 배틀 2',
 'ROBLOX',
 '천무',
 'Fish Go.io',
 '탱글 마스터 3D',
 '워드퍼즐 - 단어 게임! 재미있는 단어 퍼즐',
 'Brain Test 2: 교묘한 이야기',
 'FIFA Mobile',
 'AFK 아레나',
 'Tower Run',
 'Spiral Roll',
 '랜덤 다이스(Random Dice): PvP 디펜스',
 '무승부등산가 - Draw Climber',
 '스톤에이지 월드',
 '관도:삼국지',
 '브롤스타즈',
 '꿈의 정원 (Gardenscapes)',
 'Just Draw',
 '콤배트 오브 히어로 - Combat of Hero',
 'Brain Out – 넌센스 마스터, 두뇌 트레이닝',
 'Rhythm Games',
 '히어로볼 Z',
 '1945',
 '단어호감 - 단어 지우기 게임',
 '리듬 대보검',
 '최고의 클라우드 게임 플랫폼! Netboom',
 'Tiles Hop: EDM Rush!',
 'Johnny Trigger: Sniper',
 '꿈의 집 (Homescapes)',
 'Prison Empire Tycoon - 방치형 게임',
 'Minecraft 체험판',
 '아이스크림 롤',
 '구워요!

# 장르 가져오기

각 페이지 별 장르를 넣어야 하는데, 장르는 페이지 안에 있음 -> baseUrl을 두고 각 page별로 들어가서 genre들고오기

In [11]:
url_genre = []

In [12]:
baseUrl = 'https://play.google.com'
for i in range(len(url_href)):
    print(i)
    url2 = baseUrl + url_href[i]
    html = urllib.request.urlopen(url2).read()
    soup2 = BeautifulSoup(html, 'html.parser')
    
    title_genre = soup2.find_all(class_ = 'hrTbp R8zArc')
    url_genre.append(title_genre[1].text)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

url_href_remove_duplicate에 수집된 각 url로 들어가 genre를 크롤링 해오는 과정(너무 오랜 시간이 걸림 -> 다른 해결책은?)

In [13]:
len(url_genre)

1045

In [14]:
url_genre

['Puzzle',
 'Sports',
 'Role Playing',
 'Casual',
 'Casual',
 'Racing',
 'Casual',
 'Action',
 'Action',
 'Role Playing',
 'Strategy',
 'Simulation',
 'Casual',
 'Action',
 'Casual',
 'Role Playing',
 'Casual',
 'Role Playing',
 'Simulation',
 'Casual',
 'Strategy',
 'Action',
 'Role Playing',
 'Role Playing',
 'Adventure',
 'Role Playing',
 'Action',
 'Puzzle',
 'Puzzle',
 'Trivia',
 'Sports',
 'Role Playing',
 'Arcade',
 'Arcade',
 'Strategy',
 'Arcade',
 'Role Playing',
 'Strategy',
 'Action',
 'Casual',
 'Puzzle',
 'Action',
 'Puzzle',
 'Casual',
 'Role Playing',
 'Arcade',
 'Word',
 'Music',
 'Action',
 'Music',
 'Action',
 'Casual',
 'Simulation',
 'Arcade',
 'Simulation',
 'Casual',
 'Action',
 'Arcade',
 'Trivia',
 'Puzzle',
 'Arcade',
 'Puzzle',
 'Casual',
 'Casual',
 'Arcade',
 'Casual',
 'Role Playing',
 'Puzzle',
 'Action',
 'Arcade',
 'Puzzle',
 'Adventure',
 'Role Playing',
 'Casual',
 'Arcade',
 'Casual',
 'Role Playing',
 'Role Playing',
 'Casual',
 'Card',
 'Action',
 

# 분석 및 CSV 파일 만들기

게임이름, 게임 URL, 게임 장르를 분류하고, CSV 파일로 만들기

In [15]:
pd.DataFrame({'Game title' : game_title,'Url' : url_href, 'Genre' : url_genre })

,Game title,Url,Genre
0,애니팡4,/store/apps/details?id=com.sundaytoz.kakao.ani...,Puzzle
1,마구마구 2020,/store/apps/details?id=com.netmarble.ma9bkr,Sports
2,라그나로크 오리진,/store/apps/details?id=com.gravity.ragnarokori...,Role Playing
3,라인업: 몽타주 그리기,/store/apps/details?id=com.jhulse.lineup,Casual
4,Stack Colors!,/store/apps/details?id=com.redforcegames.stack...,Casual
...,...,...,...
1040,레전드 오브 룬테라,/store/apps/details?id=com.riotgames.legendsof...,Card
1041,트리비아크랙,/store/apps/details?id=com.etermax.preguntados...,Trivia
1042,리틀빅시티2,/store/apps/details?id=com.gameloft.android.AN...,Casual
1043,스나이퍼 퓨리: 최고의 슈팅게임 - FPS,/store/apps/details?id=com.gameloft.android.AN...,Action


여러 url을 통해 title, url, genre를 수집했습니다. 여기서 중복, 필요 없는 부분이 존재해 제거해야 합니다.
* url의 필요없는 부분 제거(/store/apps/details?id= 까지는 모두 같고, 그 뒤는 com 이거나 net .. 등임, com, net 또한 장르에 영향을 끼치지 않음)
* url을 기준 열로 잡고, 중복된 row 제거

1. url의 필요없는 부분 제거

In [16]:
url_href

['/store/apps/details?id=com.sundaytoz.kakao.anipang4',
 '/store/apps/details?id=com.netmarble.ma9bkr',
 '/store/apps/details?id=com.gravity.ragnarokorigin.aos',
 '/store/apps/details?id=com.jhulse.lineup',
 '/store/apps/details?id=com.redforcegames.stack.colors',
 '/store/apps/details?id=com.nexon.kart',
 '/store/apps/details?id=com.xtt.iasurv2',
 '/store/apps/details?id=ru.galya.drawjoust',
 '/store/apps/details?id=com.dangernoodle.killerbee',
 '/store/apps/details?id=com.monawa.samblood',
 '/store/apps/details?id=com.sangoku2kr.android',
 '/store/apps/details?id=com.idleif.abyssrium',
 '/store/apps/details?id=com.hellotoon.shinvatar',
 '/store/apps/details?id=com.klab.bleach',
 '/store/apps/details?id=com.outfit7.mytalkingtomfriends',
 '/store/apps/details?id=com.ncsoft.lineagem19',
 '/store/apps/details?id=com.cg.moneybuster',
 '/store/apps/details?id=com.yoozoo.kr.snsgz2',
 '/store/apps/details?id=droidhang.twgame.restaurant',
 '/store/apps/details?id=com.neonnewcrystal.blast',
 '

In [17]:
imsi_url_href = []
clear_url_href = []
for i in range(len(url_href)):
    imsi_url_href.append(url_href[i].replace("/store/apps/details?id=com.",""))
    clear_url_href.append(imsi_url_href[i].replace("/store/apps/details?id=",""))

In [18]:
clear_url_href

['sundaytoz.kakao.anipang4',
 'netmarble.ma9bkr',
 'gravity.ragnarokorigin.aos',
 'jhulse.lineup',
 'redforcegames.stack.colors',
 'nexon.kart',
 'xtt.iasurv2',
 'ru.galya.drawjoust',
 'dangernoodle.killerbee',
 'monawa.samblood',
 'sangoku2kr.android',
 'idleif.abyssrium',
 'hellotoon.shinvatar',
 'klab.bleach',
 'outfit7.mytalkingtomfriends',
 'ncsoft.lineagem19',
 'cg.moneybuster',
 'yoozoo.kr.snsgz2',
 'droidhang.twgame.restaurant',
 'neonnewcrystal.blast',
 'poa.kr',
 'pubg.krmobile',
 'gf.p5krkr.hwyad.google',
 'carolgames.moemoegirls',
 'roblox.client',
 'ag.lwkr',
 'whitedot.bfg',
 'rollic.tanglemaster3D',
 'wordpuzzle.kr',
 'unicostudio.braintest2new',
 'nexon.fmk',
 'lilithgame.hgame.gp.kr',
 'lawson.tower',
 'Celltop.SpiralRoll',
 'percent.royaldice',
 'appadvisory.drawclimber',
 'netmarble.stonemmogb',
 'samkokkr.gp',
 'supercell.brawlstars',
 'playrix.gardenscapes',
 'onebutton.jd',
 'peakx.stickfight.and.inter',
 'mind.quiz.brain.out',
 'game.jam.rhythm.music.tap',
 'joyc

In [19]:
df = pd.DataFrame({'Game title' : game_title,'Url' : clear_url_href, 'Genre' : url_genre })
df

,Game title,Url,Genre
0,애니팡4,sundaytoz.kakao.anipang4,Puzzle
1,마구마구 2020,netmarble.ma9bkr,Sports
2,라그나로크 오리진,gravity.ragnarokorigin.aos,Role Playing
3,라인업: 몽타주 그리기,jhulse.lineup,Casual
4,Stack Colors!,redforcegames.stack.colors,Casual
...,...,...,...
1040,레전드 오브 룬테라,riotgames.legendsofruneterra,Card
1041,트리비아크랙,etermax.preguntados.lite,Trivia
1042,리틀빅시티2,gameloft.android.ANMP.GloftL2HM,Casual
1043,스나이퍼 퓨리: 최고의 슈팅게임 - FPS,gameloft.android.ANMP.GloftFWHM,Action


2. url을 기준으로 중복된 row 제거

In [20]:
Training_set = df.drop_duplicates('Url', keep = 'first')

In [21]:
Training_set.reset_index(inplace = True)
Training_set.drop('index', axis = 1, inplace = True)
Training_set

C:\Users\yah12\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Game title,Url,Genre
0,애니팡4,sundaytoz.kakao.anipang4,Puzzle
1,마구마구 2020,netmarble.ma9bkr,Sports
2,라그나로크 오리진,gravity.ragnarokorigin.aos,Role Playing
3,라인업: 몽타주 그리기,jhulse.lineup,Casual
4,Stack Colors!,redforcegames.stack.colors,Casual
...,...,...,...
825,레전드 오브 룬테라,riotgames.legendsofruneterra,Card
826,트리비아크랙,etermax.preguntados.lite,Trivia
827,리틀빅시티2,gameloft.android.ANMP.GloftL2HM,Casual
828,스나이퍼 퓨리: 최고의 슈팅게임 - FPS,gameloft.android.ANMP.GloftFWHM,Action


3. csv 파일 변환

In [22]:
Training_set.to_csv('Training_set',index = False, header = False)

이상 구글 플레이스토어 크롤링 후 Training_set 만들기 종료

추후 수정 필요
1. Training_set에 자세히 보기 추가
2. Game title을 모두 영어로 적고싶음(몇 번 시도했는데 실패)
  * google play store에 들어가서 chrome을 영어로 번역 후 class를 들고와 봤으나 실패 -> 어떻게 해야할까?
3. google play store가 어플을 최대 200개만 보여줘서 생각보다 작업이 오래 걸림 -> 해결책은?
4. 각 genre를 들고오는 과정이 매우 복잡 -> 해결책은?